# Food Swamp Analysis in Contra Costa & Solano Counties

In [ ]:
!pip install geocoder==1.38.1


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install geopy==2.4.0


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# include import statements here
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.style.use('fivethirtyeight')

import zipfile
from pathlib import Path

import os
import plotly

from pathlib import Path
data_dir = Path('.')
data_dir.mkdir(exist_ok = True)
file_path = data_dir / Path('data.zip')
dest_path = file_path

import folium
from folium import plugins
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

import geocoder
from geopy import *

from geopy.geocoders import Nominatim

## Fast Food

In [ ]:
fast_food_uncleaned = pd.read_csv("updated_fast_food_data.csv")
fast_food_uncleaned

,url,name,rating,price,category,location
0,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.google.com/maps/place/Five+Guys/da...,Five Guys,4.2,$$,Fast Food,190 Alamo Plaza
2,https://www.google.com/maps/place/Taco+Bell/da...,Taco Bell,3.7,$,Fast Food,420 Diablo Rd Suite 420
3,https://www.google.com/maps/place/Fosters+Free...,Fosters Freeze,4.3,$,Fast Food,180 Hartz Ave
4,https://www.google.com/maps/place/Jersey+Mike'...,Jersey Mike's Subs,4.5,$,Fast Food,301 Hartz Ave Ste. 100
...,...,...,...,...,...,...
1035,https://www.google.com/maps/place/Taco+Bell/da...,Taco Bell,3.7,$,Mexican-inspired fast-food chain,703 Matsumoto Lane
1036,https://www.google.com/maps/place/Burger+King/...,Burger King,3.2,$,Fast-food chain for grilled burgers,"701 Matsumoto Lane, Suite A, County Rd 90"
1037,https://www.google.com/maps/place/Subway/data=...,Subway,4.1,$,Build-your-own sandwich chain,180 E Grant Ave
1038,https://www.google.com/maps/place/Chuy's+Taque...,Chuy's Taqueria,4.3,$,Casual eatery with hearty Mexican fare,208 Railroad Ave


In [ ]:
names_to_drop = ['Applebee’s', 'Starbucks', 'Lucky', 'Panera bread', 'Black bear diner ']

fast_food = fast_food_uncleaned.copy()

for name in names_to_drop:
    fast_food = fast_food[~(fast_food_uncleaned['name'] == name)]


fast_food = fast_food.dropna(subset=['name'])
fast_food

/tmp/ipykernel_642/769507919.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fast_food = fast_food[~(fast_food_uncleaned['name'] == name)]


,url,name,rating,price,category,location
1,https://www.google.com/maps/place/Five+Guys/da...,Five Guys,4.2,$$,Fast Food,190 Alamo Plaza
2,https://www.google.com/maps/place/Taco+Bell/da...,Taco Bell,3.7,$,Fast Food,420 Diablo Rd Suite 420
3,https://www.google.com/maps/place/Fosters+Free...,Fosters Freeze,4.3,$,Fast Food,180 Hartz Ave
4,https://www.google.com/maps/place/Jersey+Mike'...,Jersey Mike's Subs,4.5,$,Fast Food,301 Hartz Ave Ste. 100
5,https://www.google.com/maps/place/Chipotle+Mex...,Chipotle Mexican Grill,3.2,$,Fast Food,33 Railroad Ave
...,...,...,...,...,...,...
1035,https://www.google.com/maps/place/Taco+Bell/da...,Taco Bell,3.7,$,Mexican-inspired fast-food chain,703 Matsumoto Lane
1036,https://www.google.com/maps/place/Burger+King/...,Burger King,3.2,$,Fast-food chain for grilled burgers,"701 Matsumoto Lane, Suite A, County Rd 90"
1037,https://www.google.com/maps/place/Subway/data=...,Subway,4.1,$,Build-your-own sandwich chain,180 E Grant Ave
1038,https://www.google.com/maps/place/Chuy's+Taque...,Chuy's Taqueria,4.3,$,Casual eatery with hearty Mexican fare,208 Railroad Ave


In [ ]:
len(fast_food_uncleaned)

In [ ]:
len(fast_food)

1022

In [ ]:
lat = r'\!3d(-?\d{2,3}\.\d+)!'
lon = r'\!4d(-?\d{2,3}\.\d+)!'

fast_food['lat'] = fast_food['url'].str.extract(lat)
fast_food['lon'] = fast_food['url'].str.extract(lon)

fast_food

,url,name,rating,price,category,location,lat,lon
1,https://www.google.com/maps/place/Five+Guys/da...,Five Guys,4.2,$$,Fast Food,190 Alamo Plaza,37.8513907,-122.0352805
2,https://www.google.com/maps/place/Taco+Bell/da...,Taco Bell,3.7,$,Fast Food,420 Diablo Rd Suite 420,37.8235609,-121.9929895
3,https://www.google.com/maps/place/Fosters+Free...,Fosters Freeze,4.3,$,Fast Food,180 Hartz Ave,37.823741,-122.001777
4,https://www.google.com/maps/place/Jersey+Mike'...,Jersey Mike's Subs,4.5,$,Fast Food,301 Hartz Ave Ste. 100,37.8223604,-122.0011072
5,https://www.google.com/maps/place/Chipotle+Mex...,Chipotle Mexican Grill,3.2,$,Fast Food,33 Railroad Ave,37.8231593,-122.0040949
...,...,...,...,...,...,...,...,...
1035,https://www.google.com/maps/place/Taco+Bell/da...,Taco Bell,3.7,$,Mexican-inspired fast-food chain,703 Matsumoto Lane,38.5317559,-121.9558002
1036,https://www.google.com/maps/place/Burger+King/...,Burger King,3.2,$,Fast-food chain for grilled burgers,"701 Matsumoto Lane, Suite A, County Rd 90",38.5311337,-121.9555683
1037,https://www.google.com/maps/place/Subway/data=...,Subway,4.1,$,Build-your-own sandwich chain,180 E Grant Ave,38.5266661,-121.9651382
1038,https://www.google.com/maps/place/Chuy's+Taque...,Chuy's Taqueria,4.3,$,Casual eatery with hearty Mexican fare,208 Railroad Ave,38.5213435,-121.9687098


In [ ]:
fast_food['type'] = 'unhealthy'
fast_food

In [ ]:
# Create a sample map centered around the geographical center of Contra Costa County
center_address = '595 Center Ave, Martinez, CA 94553'

# get location information for address
locator = Nominatim(user_agent="myGeocoder")
center = locator.geocode(center_address)

# map
county_map = folium.Map(location=[center.latitude, center.longitude], zoom_start=11)

# display map
county_map

GeocoderInsufficientPrivileges: Non-successful status code 403

## Convenience Stores & Gas Stations

In [ ]:
convenience_stores_uncleaned = pd.read_csv("/work/contra-costa-solano-convenience-stores - convenience stores.csv")
convenience_stores_uncleaned

,url,name,rating,price,category,address
0,https://www.google.com/maps/place/Dash/data=!4...,Dash,3.6,$,Convenience store,1502 Discovery Bay Blvd
1,https://www.google.com/maps/place/Country+Junc...,Country Junction Chevron,1.8,$,Convenience store,2995 Taylor Ln
2,https://www.google.com/maps/place/Valero/data=...,Valero,5,$,Gas station,1502 Discovery Bay Blvd
3,https://www.google.com/maps/place/Michael%E2%8...,Michael’s market,4.4,$,Gas station,1502 Discovery Bay Blvd
4,https://www.google.com/maps/place/Chevron/data...,Chevron,1.5,$,Convenience store,2995 Taylor Ln
...,...,...,...,...,...,...
1080,https://www.google.com/maps/place/Jiffy+Lube/d...,Jiffy Lube,3.9,NaN,Oil change service,4300 Sonoma Blvd
1081,https://www.google.com/maps/place/U.S.+Bank+AT...,U.S. Bank ATM,3.5,NaN,ATM,1200 Tennessee St
1082,https://www.google.com/maps/place/Jiffy+Lube/d...,Jiffy Lube,4.6,NaN,Oil change service,2026 Columbus Pkwy
1083,https://www.google.com/maps/place/Stop+N+Save+...,Stop N Save Inc,2.3,NaN,Convenience store,2200 Sacramento St


In [ ]:
orig_num_rows = len(convenience_stores_uncleaned)
orig_num_rows

1085

In [ ]:
convenience_stores = convenience_stores_uncleaned.copy()

names_to_drop = ['Safeway', 'Rite Aid', 'Starbucks', 'CVS Photo', 'Walgreens', 'Lowes', 'Ulta Beauty', 'Tractor Supply Co.']

categories_to_drop = ['Tire shop', 'Auto repair shop', 'Oil change service', 'Bicycle shop', 
'Pet supply store', 'Pharmacy', 'Welding supply store', 'Business management consultant', 
'Office supply store', 'Cosmetics store', 'Passport photo processor', 'ATM']

for name in names_to_drop:
    convenience_stores = convenience_stores[~(convenience_stores['name'] == name)]

for category in categories_to_drop:
    convenience_stores = convenience_stores[~(convenience_stores['category'] == category)]

convenience_stores = convenience_stores.dropna(subset=['name'])
convenience_stores

,url,name,rating,price,category,address
0,https://www.google.com/maps/place/Dash/data=!4...,Dash,3.6,$,Convenience store,1502 Discovery Bay Blvd
1,https://www.google.com/maps/place/Country+Junc...,Country Junction Chevron,1.8,$,Convenience store,2995 Taylor Ln
2,https://www.google.com/maps/place/Valero/data=...,Valero,5,$,Gas station,1502 Discovery Bay Blvd
3,https://www.google.com/maps/place/Michael%E2%8...,Michael’s market,4.4,$,Gas station,1502 Discovery Bay Blvd
4,https://www.google.com/maps/place/Chevron/data...,Chevron,1.5,$,Convenience store,2995 Taylor Ln
...,...,...,...,...,...,...
1077,https://www.google.com/maps/place/Wheel+Works/...,Wheel Works,4,NaN,Car repair and maintenance service,3560 Sonoma Blvd
1078,https://www.google.com/maps/place/Little+Maya+...,Little Maya Bakery,4.1,NaN,Bakery,985 Broadway
1079,https://www.google.com/maps/place/DolEx+Dollar...,DolEx Dollar Express,3.9,NaN,Financial institution,2828 Sonoma Blvd
1083,https://www.google.com/maps/place/Stop+N+Save+...,Stop N Save Inc,2.3,NaN,Convenience store,2200 Sacramento St


In [ ]:
# Number of rows removed 
num_rows_removed = len(convenience_stores_uncleaned) - len(convenience_stores)
num_rows_removed

132

In [ ]:
lat = r'\!3d(-?\d{2,3}\.\d+)!'
lon = r'\!4d(-?\d{2,3}\.\d+)!'

convenience_stores['lat'] = convenience_stores['url'].str.extract(lat)
convenience_stores['lon'] = convenience_stores['url'].str.extract(lon)

convenience_stores['type'] = 'unhealthy'
convenience_stores

,url,name,rating,price,category,address,lat,lon,type
0,https://www.google.com/maps/place/Dash/data=!4...,Dash,3.6,$,Convenience store,1502 Discovery Bay Blvd,37.9041895,-121.6024693,unhealthy
1,https://www.google.com/maps/place/Country+Junc...,Country Junction Chevron,1.8,$,Convenience store,2995 Taylor Ln,37.889871,-121.641906,unhealthy
2,https://www.google.com/maps/place/Valero/data=...,Valero,5,$,Gas station,1502 Discovery Bay Blvd,37.9041693,-121.6024674,unhealthy
3,https://www.google.com/maps/place/Michael%E2%8...,Michael’s market,4.4,$,Gas station,1502 Discovery Bay Blvd,37.9039929,-121.6026634,unhealthy
4,https://www.google.com/maps/place/Chevron/data...,Chevron,1.5,$,Convenience store,2995 Taylor Ln,37.889491,-121.641995,unhealthy
...,...,...,...,...,...,...,...,...,...
1077,https://www.google.com/maps/place/Wheel+Works/...,Wheel Works,4,NaN,Car repair and maintenance service,3560 Sonoma Blvd,38.1217706,-122.2546291,unhealthy
1078,https://www.google.com/maps/place/Little+Maya+...,Little Maya Bakery,4.1,NaN,Bakery,985 Broadway,38.1156561,-122.248838,unhealthy
1079,https://www.google.com/maps/place/DolEx+Dollar...,DolEx Dollar Express,3.9,NaN,Financial institution,2828 Sonoma Blvd,38.1127912,-122.2543247,unhealthy
1083,https://www.google.com/maps/place/Stop+N+Save+...,Stop N Save Inc,2.3,NaN,Convenience store,2200 Sacramento St,38.1199519,-122.2598928,unhealthy


## Map of all Unhealthy Places

In [ ]:
# plot all the locations from the dataset
county_map

for (index, row) in fast_food.iterrows():
    folium.Circle(location=[row.loc['lat'], row.loc['lon']], radius = 20,
                  color='red', fill_color='red').add_to(county_map)

for (index, row) in convenience_stores.iterrows():
    folium.Circle(location=[row.loc['lat'], row.loc['lon']], radius = 20,
                  color='purple', fill_color='purple').add_to(county_map)

# display map
county_map

## Grocery Stores  - Healthy Place

In [ ]:
grocery_uncleaned = pd.read_csv("contra-costa-solano-grocery (cleaned).csv")
grocery_uncleaned

,url,name,rating,price,category,address,description
0,https://www.google.com/maps/place/Safeway/data...,Safeway,4.1,$$,Grocery store,3365 Deer Valley Rd,NaN
1,https://www.google.com/maps/place/Raley's/data...,Raley's,4.1,$$,Grocery store,3632 Lone Tree Wy,NaN
2,https://www.google.com/maps/place/Antioch+Food...,Antioch Food Center,4.1,$$,Grocery store,111 E 18th St,NaN
3,https://www.google.com/maps/place/Lucky/data=!...,Lucky,4.2,$$,Grocery store,3190 Contra Loma Blvd,NaN
4,https://www.google.com/maps/place/County+Marke...,County Market,4.4,$$,Asian grocery store,2711 Hillcrest Ave Suite 208,NaN
...,...,...,...,...,...,...,...
833,https://www.google.com/maps/place/Tj+Mart/data...,Tj Mart,5.0,NaN,Convenience store,2400 Sacramento St,·
834,https://www.google.com/maps/place/La+Rosa+Mark...,La Rosa Market,4.7,NaN,Grocery store,738 Sonoma Blvd,·
835,https://www.google.com/maps/place/B+%26+N+Supe...,B & N Super Stop,2.0,NaN,Grocery store,701 Mini Dr,·
836,https://www.google.com/maps/place/Guanajuato+M...,Guanajuato Market Taqueria,4.4,NaN,Market,18 Laurel St,·


In [ ]:
names_to_drop = ['7-Eleven', 'Big Lots', 'CVS', 'Denny\'s']
categories_to_drop = ['Liquor store', 'Drug store', 'ATM', 'Wine store', 
'Auto parts store', 'Convenience store', 'Nut store', 'Bus stop', 'Truck stop', 
'2-star hotel', 'Bottled water supplier', 'Park', 'Ice Cream', 'Store', 'Shopping mall',
'Non-profit organization', 'Coffee shop', 'Gas station', 'Pharmacy', 'Business center',
'Restaurant', 'Drug store', 'Garden center']

grocery = grocery_uncleaned.copy()

for name in names_to_drop:
    grocery = grocery[~(grocery_uncleaned['name'] == name)]

for category in categories_to_drop:
    grocery = grocery[~(grocery['category'] == category)]

grocery

/tmp/ipykernel_94/1431052426.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  grocery = grocery[~(grocery_uncleaned['name'] == name)]


,url,name,rating,price,category,address,description
0,https://www.google.com/maps/place/Safeway/data...,Safeway,4.1,$$,Grocery store,3365 Deer Valley Rd,NaN
1,https://www.google.com/maps/place/Raley's/data...,Raley's,4.1,$$,Grocery store,3632 Lone Tree Wy,NaN
2,https://www.google.com/maps/place/Antioch+Food...,Antioch Food Center,4.1,$$,Grocery store,111 E 18th St,NaN
3,https://www.google.com/maps/place/Lucky/data=!...,Lucky,4.2,$$,Grocery store,3190 Contra Loma Blvd,NaN
4,https://www.google.com/maps/place/County+Marke...,County Market,4.4,$$,Asian grocery store,2711 Hillcrest Ave Suite 208,NaN
...,...,...,...,...,...,...,...
831,https://www.google.com/maps/place/Lola%E2%80%9...,Lola’s Market,3.9,NaN,Supermarket,5201 Sonoma Blvd #7,Latino groceries & Mexican fare
832,https://www.google.com/maps/place/Island+Pacif...,Island Pacific Seafood Market,4.1,$$,Asian grocery store,2110 Springs Rd UNIT 24,Market dedicated to Filipino groceries
834,https://www.google.com/maps/place/La+Rosa+Mark...,La Rosa Market,4.7,NaN,Grocery store,738 Sonoma Blvd,·
835,https://www.google.com/maps/place/B+%26+N+Supe...,B & N Super Stop,2.0,NaN,Grocery store,701 Mini Dr,·


In [ ]:
lat_match = r'\!3d(-?\d{2,3}\.\d+)!'
lon_match = r'\!4d(-?\d{2,3}\.\d+)!'

grocery['lat'] = grocery['url'].str.extract(lat_match)
grocery['lon'] = grocery['url'].str.extract(lon_match)

grocery['type'] = 'healthy'
grocery

,url,name,rating,price,category,address,description,lat,lon,type
0,https://www.google.com/maps/place/Safeway/data...,Safeway,4.1,$$,Grocery store,3365 Deer Valley Rd,NaN,37.9877653,-121.7840092,healthy
1,https://www.google.com/maps/place/Raley's/data...,Raley's,4.1,$$,Grocery store,3632 Lone Tree Wy,NaN,37.9856548,-121.807338,healthy
2,https://www.google.com/maps/place/Antioch+Food...,Antioch Food Center,4.1,$$,Grocery store,111 E 18th St,NaN,38.0056797,-121.801459,healthy
3,https://www.google.com/maps/place/Lucky/data=!...,Lucky,4.2,$$,Grocery store,3190 Contra Loma Blvd,NaN,37.9905367,-121.8238187,healthy
4,https://www.google.com/maps/place/County+Marke...,County Market,4.4,$$,Asian grocery store,2711 Hillcrest Ave Suite 208,NaN,37.9947007,-121.7815807,healthy
...,...,...,...,...,...,...,...,...,...,...
831,https://www.google.com/maps/place/Lola%E2%80%9...,Lola’s Market,3.9,NaN,Supermarket,5201 Sonoma Blvd #7,Latino groceries & Mexican fare,38.1549778,-122.255378,healthy
832,https://www.google.com/maps/place/Island+Pacif...,Island Pacific Seafood Market,4.1,$$,Asian grocery store,2110 Springs Rd UNIT 24,Market dedicated to Filipino groceries,38.1046099,-122.2171251,healthy
834,https://www.google.com/maps/place/La+Rosa+Mark...,La Rosa Market,4.7,NaN,Grocery store,738 Sonoma Blvd,·,38.0887295,-122.2440876,healthy
835,https://www.google.com/maps/place/B+%26+N+Supe...,B & N Super Stop,2.0,NaN,Grocery store,701 Mini Dr,·,38.15272,-122.2490147,healthy


In [ ]:
# Center latitude and longitude
# 38.113642, -122.067598

# map
county_map = folium.Map(location=[38.113642, -122.067598], zoom_start=10)


for (index, row) in grocery.iterrows():
    folium.Circle(location=[row.loc['lat'], row.loc['lon']], radius = 20,
                  color='blue', fill_color='blue').add_to(county_map)


# display map
county_map

## Map of Unhealthy and Healthy options overlayed

In [ ]:
# Center latitude and longitude
# 38.113642, -122.067598

# map
county_map = folium.Map(location=[38.113642, -122.067598], zoom_start=10)


for (index, row) in grocery.iterrows():
    folium.Circle(location=[row.loc['lat'], row.loc['lon']], radius = 20,
                  color='blue', fill_color='blue').add_to(county_map)

for (index, row) in fast_food.iterrows():
    folium.Circle(location=[row.loc['lat'], row.loc['lon']], radius = 20,
                  color='red', fill_color='red').add_to(county_map)

for (index, row) in convenience_stores.iterrows():
    folium.Circle(location=[row.loc['lat'], row.loc['lon']], radius = 20,
                  color='purple', fill_color='purple').add_to(county_map)

# display map
county_map

In [ ]:
# Experiment with converting longitude and latitude to addresses to reverse geocode
# https://datascientyst.com/reverse-geocoding-latitude-longitude-city-country-python-pandas/


# Initialize Nominatim API
geolocator = Nominatim(user_agent="geoapiExercises")

grocery_copy = grocery.copy()
grocery_copy = grocery_copy.iloc[0:50]
grocery_copy

grocery_copy['location'] = pd.Series()

# 38.113642, -122.067598

location = geolocator.reverse(grocery_copy['lat'][0] + ", " + grocery_copy['lon'][0])
print(location.raw.get('address').get('city'))

# for (index, row) in grocery_copy.iterrows():
#     lat_and_lon = str(row.loc['lat'])+', '+str(row.loc['lon'])
#     location = geolocator.reverse(lat_and_lon)
#     row.loc['location'] = location.raw.get('address').get('city')
#     print(row.loc['location'])


/tmp/ipykernel_94/3976869263.py:13: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  grocery_copy['location'] = pd.Series()
Antioch


In [ ]:
import geopy.distance

coords_1 = (float(grocery_copy['lat'][0]), float(grocery_copy['lon'][0]))
coords_2 = (52.406374, 16.9251681)

geopy.distance.geodesic(coords_1, coords_2).mi

5731.415037445597

Question I am trying to answer:

For each location, determine the number of stores of other categories in < 2 mile radius. Create a ratio of #unhealthy / #healthy. Create heat map of risk level of "Food Swampiness".

The number of grocery stores in a 5 mile radius to the center is __ and the number of fast food places / convenience stores in a 5 mile radius to the center is __. 

Therefore, the ratio between healthy and unhealthy options is x/y, and it is [considered a food swamp] OR [not considered a food swamp].

In [ ]:
heat_map = folium.Map(location=[38.113642, -122.067598], zoom_start=10)

In [ ]:
combined_data = grocery

In [ ]:
# lat = grocery['LATITUDE'].tolist()
# lon = cleaned_lights['LONGITUDE'].tolist()
# risk_factor = cleaned_lights['risk_factor'].tolist()
# locations_and_risk_factor = []

# for i in range(len(lat)):
#    locations_and_risk_factor += [ [lat[i], lon[i], risk_factor[i]] ]
   
# risk_factor_map = folium.Map(location = [rsf_lat, rsf_long], zoom_start=13.5, width = 900, height= 600, control_scale=True)
# HeatMap(data=locations_and_severity).add_to(risk_factor_map)
# folium.LayerControl().add_to(risk_factor_map)

# # display map
# risk_factor_map

In [ ]:
#grocery
#drop description 
grocery_will_be_merged = grocery.drop('description', axis = 1)
grocery_will_be_merged

,url,name,rating,price,category,address,lat,lon,type
0,https://www.google.com/maps/place/Safeway/data...,Safeway,4.1,$$,Grocery store,3365 Deer Valley Rd,37.9877653,-121.7840092,healthy
1,https://www.google.com/maps/place/Raley's/data...,Raley's,4.1,$$,Grocery store,3632 Lone Tree Wy,37.9856548,-121.807338,healthy
2,https://www.google.com/maps/place/Antioch+Food...,Antioch Food Center,4.1,$$,Grocery store,111 E 18th St,38.0056797,-121.801459,healthy
3,https://www.google.com/maps/place/Lucky/data=!...,Lucky,4.2,$$,Grocery store,3190 Contra Loma Blvd,37.9905367,-121.8238187,healthy
4,https://www.google.com/maps/place/County+Marke...,County Market,4.4,$$,Asian grocery store,2711 Hillcrest Ave Suite 208,37.9947007,-121.7815807,healthy
...,...,...,...,...,...,...,...,...,...
831,https://www.google.com/maps/place/Lola%E2%80%9...,Lola’s Market,3.9,NaN,Supermarket,5201 Sonoma Blvd #7,38.1549778,-122.255378,healthy
832,https://www.google.com/maps/place/Island+Pacif...,Island Pacific Seafood Market,4.1,$$,Asian grocery store,2110 Springs Rd UNIT 24,38.1046099,-122.2171251,healthy
834,https://www.google.com/maps/place/La+Rosa+Mark...,La Rosa Market,4.7,NaN,Grocery store,738 Sonoma Blvd,38.0887295,-122.2440876,healthy
835,https://www.google.com/maps/place/B+%26+N+Supe...,B & N Super Stop,2.0,NaN,Grocery store,701 Mini Dr,38.15272,-122.2490147,healthy


In [ ]:
#fast_food
#change location to address 
fast_food_will_be_merged = fast_food.rename(columns= {'location': 'address'})
fast_food_will_be_merged.head()

,url,name,rating,price,category,address,lat,lon,type
1,https://www.google.com/maps/place/Five+Guys/da...,Five Guys,4.2,$$,Fast Food,190 Alamo Plaza,37.8513907,-122.0352805,unhealthy
2,https://www.google.com/maps/place/Taco+Bell/da...,Taco Bell,3.7,$,Fast Food,420 Diablo Rd Suite 420,37.8235609,-121.9929895,unhealthy
3,https://www.google.com/maps/place/Fosters+Free...,Fosters Freeze,4.3,$,Fast Food,180 Hartz Ave,37.823741,-122.001777,unhealthy
4,https://www.google.com/maps/place/Jersey+Mike'...,Jersey Mike's Subs,4.5,$,Fast Food,301 Hartz Ave Ste. 100,37.8223604,-122.0011072,unhealthy
5,https://www.google.com/maps/place/Chipotle+Mex...,Chipotle Mexican Grill,3.2,$,Fast Food,33 Railroad Ave,37.8231593,-122.0040949,unhealthy


In [ ]:
convenience_stores.head()

,url,name,rating,price,category,address,lat,lon,type
0,https://www.google.com/maps/place/Dash/data=!4...,Dash,3.6,$,Convenience store,1502 Discovery Bay Blvd,37.9041895,-121.6024693,unhealthy
1,https://www.google.com/maps/place/Country+Junc...,Country Junction Chevron,1.8,$,Convenience store,2995 Taylor Ln,37.889871,-121.641906,unhealthy
2,https://www.google.com/maps/place/Valero/data=...,Valero,5,$,Gas station,1502 Discovery Bay Blvd,37.9041693,-121.6024674,unhealthy
3,https://www.google.com/maps/place/Michael%E2%8...,Michael’s market,4.4,$,Gas station,1502 Discovery Bay Blvd,37.9039929,-121.6026634,unhealthy
4,https://www.google.com/maps/place/Chevron/data...,Chevron,1.5,$,Convenience store,2995 Taylor Ln,37.889491,-121.641995,unhealthy


In [ ]:
combined_df = pd.concat( [convenience_stores,fast_food_will_be_merged,grocery_will_be_merged], ignore_index=True )


In [ ]:
combined_df.to_csv('Combined_data.csv', index = False)


In [ ]:
import geopandas as gp


In [ ]:
spa = gp.read_file('/work/ca_2020_census_tract 2/tl_2020_06_tract.shp')
print(spa.head())

  STATEFP COUNTYFP TRACTCE        GEOID     NAME              NAMELSAD  MTFCC  \
0      06      029  004402  06029004402    44.02    Census Tract 44.02  G5020   
1      06      047  000802  06047000802     8.02     Census Tract 8.02  G5020   
2      06      085  501402  06085501402  5014.02  Census Tract 5014.02  G5020   
3      06      005  000102  06005000102     1.02     Census Tract 1.02  G5020   
4      06      029  004901  06029004901    49.01    Census Tract 49.01  G5020   

  FUNCSTAT      ALAND   AWATER     INTPTLAT      INTPTLON  \
0        S    1865739        0  +35.5917343  -119.3337642   
1        S    2321653        0  +37.3433591  -120.5854604   
2        S     522620        0  +37.3453341  -121.8665169   
3        S  456204155  7940832  +38.5496993  -120.2509786   
4        S    1459379        0  +35.7837173  -119.2462266   

                                            geometry  
0  POLYGON ((-119.33828 35.58143, -119.33827 35.5...  
1  POLYGON ((-120.59599 37.34121, -1

In [ ]:
combined_df['lon'] = combined_df['lon'].astype('float64')

In [ ]:
combined_df['lat'] = combined_df['lat'].astype('float64')

In [ ]:
combined_df.head()

,url,name,rating,price,category,address,lat,lon,type
0,https://www.google.com/maps/place/Dash/data=!4...,Dash,3.6,$,Convenience store,1502 Discovery Bay Blvd,37.904190,-121.602469,unhealthy
1,https://www.google.com/maps/place/Country+Junc...,Country Junction Chevron,1.8,$,Convenience store,2995 Taylor Ln,37.889871,-121.641906,unhealthy
2,https://www.google.com/maps/place/Valero/data=...,Valero,5,$,Gas station,1502 Discovery Bay Blvd,37.904169,-121.602467,unhealthy
3,https://www.google.com/maps/place/Michael%E2%8...,Michael’s market,4.4,$,Gas station,1502 Discovery Bay Blvd,37.903993,-121.602663,unhealthy
4,https://www.google.com/maps/place/Chevron/data...,Chevron,1.5,$,Convenience store,2995 Taylor Ln,37.889491,-121.641995,unhealthy


In [ ]:
g_combined_df =  gp.GeoDataFrame(combined_df, geometry=gp.points_from_xy(combined_df.lon, combined_df.lat))

In [ ]:
g_combined_df.head()

,url,name,rating,price,category,address,lat,lon,type,geometry
0,https://www.google.com/maps/place/Dash/data=!4...,Dash,3.6,$,Convenience store,1502 Discovery Bay Blvd,37.904190,-121.602469,unhealthy,POINT (-121.60247 37.90419)
1,https://www.google.com/maps/place/Country+Junc...,Country Junction Chevron,1.8,$,Convenience store,2995 Taylor Ln,37.889871,-121.641906,unhealthy,POINT (-121.64191 37.88987)
2,https://www.google.com/maps/place/Valero/data=...,Valero,5,$,Gas station,1502 Discovery Bay Blvd,37.904169,-121.602467,unhealthy,POINT (-121.60247 37.90417)
3,https://www.google.com/maps/place/Michael%E2%8...,Michael’s market,4.4,$,Gas station,1502 Discovery Bay Blvd,37.903993,-121.602663,unhealthy,POINT (-121.60266 37.90399)
4,https://www.google.com/maps/place/Chevron/data...,Chevron,1.5,$,Convenience store,2995 Taylor Ln,37.889491,-121.641995,unhealthy,POINT (-121.64199 37.88949)


In [ ]:
g_combined_df['spa_names'] = ''
for row in g_combined_df.index:
    for name in spa.index:
        if spa['geometry'][name].contains(g_combined_df['geometry'][row]):
            g_combined_df['spa_names'][row] = spa['NAME'][name]
            break
g_combined_df.head()

/tmp/ipykernel_94/3221805335.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g_combined_df['spa_names'][row] = spa['NAME'][name]


,url,name,rating,price,category,address,lat,lon,type,geometry,spa_names
0,https://www.google.com/maps/place/Dash/data=!4...,Dash,3.6,$,Convenience store,1502 Discovery Bay Blvd,37.904190,-121.602469,unhealthy,POINT (-121.60247 37.90419),3040.04
1,https://www.google.com/maps/place/Country+Junc...,Country Junction Chevron,1.8,$,Convenience store,2995 Taylor Ln,37.889871,-121.641906,unhealthy,POINT (-121.64191 37.88987),3040.02
2,https://www.google.com/maps/place/Valero/data=...,Valero,5,$,Gas station,1502 Discovery Bay Blvd,37.904169,-121.602467,unhealthy,POINT (-121.60247 37.90417),3040.04
3,https://www.google.com/maps/place/Michael%E2%8...,Michael’s market,4.4,$,Gas station,1502 Discovery Bay Blvd,37.903993,-121.602663,unhealthy,POINT (-121.60266 37.90399),3040.04
4,https://www.google.com/maps/place/Chevron/data...,Chevron,1.5,$,Convenience store,2995 Taylor Ln,37.889491,-121.641995,unhealthy,POINT (-121.64199 37.88949),3040.02


In [ ]:
g_combined_df = g_combined_df.rename(columns={'spa_names': 'census_tract'})

In [ ]:
# want to export g_combined_df as csv 
csv_file = g_combined_df.to_csv(r'/work/master_cleaned_data.csv')
csv_file

In [ ]:
g_combined_df.head()

,url,name,rating,price,category,address,lat,lon,type,geometry,census_tract
0,https://www.google.com/maps/place/Dash/data=!4...,Dash,3.6,$,Convenience store,1502 Discovery Bay Blvd,37.904190,-121.602469,unhealthy,POINT (-121.60247 37.90419),3040.04
1,https://www.google.com/maps/place/Country+Junc...,Country Junction Chevron,1.8,$,Convenience store,2995 Taylor Ln,37.889871,-121.641906,unhealthy,POINT (-121.64191 37.88987),3040.02
2,https://www.google.com/maps/place/Valero/data=...,Valero,5,$,Gas station,1502 Discovery Bay Blvd,37.904169,-121.602467,unhealthy,POINT (-121.60247 37.90417),3040.04
3,https://www.google.com/maps/place/Michael%E2%8...,Michael’s market,4.4,$,Gas station,1502 Discovery Bay Blvd,37.903993,-121.602663,unhealthy,POINT (-121.60266 37.90399),3040.04
4,https://www.google.com/maps/place/Chevron/data...,Chevron,1.5,$,Convenience store,2995 Taylor Ln,37.889491,-121.641995,unhealthy,POINT (-121.64199 37.88949),3040.02


In [ ]:
# we want 
by_census_tract = g_combined_df.groupby('census_tract').apply(lambda subdf: len(subdf[subdf['type'] == 'healthy']) / len(subdf))
by_census_tract = by_census_tract.to_frame().reset_index()
by_census_tract = by_census_tract.rename(columns={0: 'proportion_healthy'})
by_census_tract['proportion_healthy'] = np.round(by_census_tract['proportion_healthy'] * 100, 2)
by_census_tract['is_swamp'] = by_census_tract['proportion_healthy'] <= 25

by_census_tract

,census_tract,proportion_healthy,is_swamp
0,,0.0,True
1,105.05,100.0,False
2,105.10,100.0,False
3,106.02,100.0,False
4,106.05,100.0,False
...,...,...,...
278,4507.43,0.0,True
279,4507.44,0.0,True
280,4507.50,0.0,True
281,98,100.0,False


In [ ]:
# Number of census tracts that are food swamps
num_food_swamps = sum(by_census_tract['is_swamp'])
num_food_swamps

159

In [ ]:
# Proportion of food swamps
proportion_food_swamps = num_food_swamps / len(by_census_tract)
proportion_food_swamps

0.5618374558303887

In [ ]:
len(by_census_tract[by_census_tract['proportion_healthy'] < 50])

197

In [ ]:
len(by_census_tract['proportion_healthy'] < 50) / len(by_census_tract['proportion_healthy'] < 50)

1.0

In [ ]:
len(by_census_tract['proportion_healthy'])

283

Therefore, we can conclude that 56% of the census tracts in Contra Costa-Solano are food swamps.

In [ ]:
by_census_tract_csv = by_census_tract.to_csv(r'/work/by_census_tract.csv')
by_census_tract_csv

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=008b30f3-1127-48f8-a4b4-f5cb6476cfc1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>